In [ ]:
#! pip install openai

In [ ]:
# Import the openai package
from openai import OpenAI

# add your API key here
client = OpenAI(api_key='')

In [ ]:
# create a base prompt that will be used for all questions
base_prompt = """From the artwork title below, give me the list of:
- object named entity
- location named entity
- person named entity
- genre named entity
- miscellaneous named entity.
Format the output in json with the following keys:
- OBJECT for organization named entity
- LOCATION for location named entity
- PERSON for person named entity
- GENRE for genre named entity
- MISCELLANEOUS for miscellaneous named entity.
Sentence below:
"""

In [ ]:
# set model parameters
openai_query_params = {
    "model": "gpt-3.5-turbo",
    "temperature": 0,
    "max_tokens": 1024
}

# define function to ask openai
def ask_openai(prompt: str, base_prompt=base_prompt, openai_query_params=openai_query_params) -> str:
    response = client.chat.completions.create(
        messages=[
        {
            "role": "system", 
            "content": "You are a smart and intelligent Named Entity Recognition (NER) system. I will provide you the definition of the entities you need to extract, the sentence from where your extract the entities and the output format."
        },
        {
            "role": "user", 
            "content": base_prompt + prompt
        }        
    ],
        **openai_query_params
    )
    
    return(response.choices[0].message.content)


In [ ]:
# example 
example_text = "Vanderbilt University is a private research university in Nashville, Tennessee. It was founded in 1873."
print(ask_openai(example_text))

In [ ]:
# custom input
input_text = " "
print(ask_openai(input_text))